In [33]:
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf

In [34]:
path = 'E:\\IITGN_Project\\autoencoder\\data\\faces\\'

In [35]:
train_datagen = ImageDataGenerator(rescale=1. / 255)


import os
from PIL import Image

faces_path = 'E:\\IITGN_Project\\autoencoder\\data\\faces\\'
images = []
for img_file in os.listdir(faces_path):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        img = load_img(os.path.join(faces_path, img_file), target_size=(256, 256))
        img_array = img_to_array(img) / 255.0 
        images.append(img_array)

train = [np.array(images)]  
print(f"Loaded {len(images)} images")

Loaded 150 images


##### by iterating on each image, we convert the RGB to Lab. Think of LAB image as a grey image in L channel and all color info stored in A and B channels. The input to the network will be the L channel, so we assign L channel to X vector. And assign A and B to Y.

In [36]:
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(150, 256, 256, 1)
(150, 256, 256, 2)


In [37]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_39 (Conv2D)              │ (None, 128, 128, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 32, 32, 256)    │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 32, 32, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_9 (UpSampling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_10 (UpSampling2D) │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 128, 128, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_50 (Conv2D)              │ (None, 128, 128, 16)   │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 128, 128, 2)    │           290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_11 (UpSampling2D) │ (None, 256, 256, 2)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,219,410 (23.73 MB)

 Trainable params: 6,219,410 (23.73 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]


model.fit(X, Y, 
          validation_split=0.1, 
          epochs=50, 
          batch_size=4,  
          callbacks=callbacks,
          verbose=1)

model.save('colorize_autoencoder.keras')

Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.4996 - loss: 0.0061 - val_accuracy: 0.6098 - val_loss: 0.0048 - learning_rate: 0.0010
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.6017 - loss: 0.0076 - val_accuracy: 0.5955 - val_loss: 0.0052 - learning_rate: 0.0010
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 33s 984ms/step - accuracy: 0.5764 - loss: 0.0074 - val_accuracy: 0.5654 - val_loss: 0.0049 - learning_rate: 0.0010
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 33s 947ms/step - accuracy: 0.5789 - loss: 0.0066 - val_accuracy: 0.5626 - val_loss: 0.0057 - learning_rate: 0.0010
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 31s 910ms/step - accuracy: 0.5319 - loss: 0.0066 - val_accuracy: 0.5597 - val_loss: 0.0048 - learning_rate: 0.0010
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.5905 - loss: 0.0063 - val_accuracy: 0.5618 - val_loss: 0.0045 - learning_rate: 0.0010
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 32s 937ms/step - accuracy: 0.5831 - loss: 0.0066 - va

In [43]:
tf.keras.models.load_model(
    r'E:\IITGN_Project\autoencoder\colorize_autoencoder.keras',
    custom_objects=None,
    compile=True)

<Sequential name=sequential_3, built=True>

In [44]:
img1_color=[]


In [47]:
img1=img_to_array(load_img(r'E:\IITGN_Project\autoencoder\1 (6).jpg'))
img1 = resize(img1 ,(256,256))
img1_color.append(img1)
img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))

In [ ]:
output1 = model.predict(img1_color)
output1 = output1*128


result = np.zeros((256, 256, 3), dtype=float)
result[:,:,0] = img1_color[0][:,:,0] 
result[:,:,1:] = output1[0] 


result[:,:,0] = np.clip(result[:,:,0], 0, 100) 
result[:,:,1:] = np.clip(result[:,:,1:], -127, 127) 

rgb_result = lab2rgb(result)
rgb_result = (rgb_result * 255).astype(np.uint8)  #

from PIL import Image as PILImage
PILImage.fromarray(rgb_result).save("result.png")
print("Image saved as result.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Image saved as result.png
